In [6]:
import os
from math import *
from time import perf_counter_ns
import gc
from random import choice
os.chdir('tsp_dataset')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'tsp_dataset'

# Parse input 

In [ ]:
def parse(fileaddress):
    CoOrdinates = {}
    with open(fileaddress) as f:
        lines = f.readlines()
    lines[0] = ': '.join(lines[0].split(' : '))
    Name = lines[0].split(" ")[1].strip()
    lines[4] = ': '.join(lines[4].split(' : '))
    EWT = lines[4].split(" ")[1].strip()
    lines[3] = ': '.join(lines[3].split(' : '))
    Dimension = lines[3].split(" ")[1].strip()
    flag = False
    for line in lines:
        line = ' '.join(line.split())
        if line.strip().split(" ")[0] == '1':
            flag = True
        elif flag == False:
            continue
        if line.strip().split(" ")[0] == 'EOF':
            break
        n,x,y = line.strip().split(" ")
        CoOrdinates[n] = (x,y)
    return Name, EWT, Dimension, CoOrdinates

<p>we save the coordinates in a dictionary with the vertex ID as the key and a (x,y) tuple as the data</p>

In [7]:
NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates  = parse('berlin52.tsp')
print(f"Data set name: {NAME}\nType: {EDGE_WEIGHT_TYPE}\nDimension: {DIMENSION}\nCoordinates: {Coordinates}")

Data set name: berlin52
Type: EUC_2D
Dimension: 52
Coordinates: {'1': ('565.0', '575.0'), '2': ('25.0', '185.0'), '3': ('345.0', '750.0'), '4': ('945.0', '685.0'), '5': ('845.0', '655.0'), '6': ('880.0', '660.0'), '7': ('25.0', '230.0'), '8': ('525.0', '1000.0'), '9': ('580.0', '1175.0'), '10': ('650.0', '1130.0'), '11': ('1605.0', '620.0'), '12': ('1220.0', '580.0'), '13': ('1465.0', '200.0'), '14': ('1530.0', '5.0'), '15': ('845.0', '680.0'), '16': ('725.0', '370.0'), '17': ('145.0', '665.0'), '18': ('415.0', '635.0'), '19': ('510.0', '875.0'), '20': ('560.0', '365.0'), '21': ('300.0', '465.0'), '22': ('520.0', '585.0'), '23': ('480.0', '415.0'), '24': ('835.0', '625.0'), '25': ('975.0', '580.0'), '26': ('1215.0', '245.0'), '27': ('1320.0', '315.0'), '28': ('1250.0', '400.0'), '29': ('660.0', '180.0'), '30': ('410.0', '250.0'), '31': ('420.0', '555.0'), '32': ('575.0', '665.0'), '33': ('1150.0', '1160.0'), '34': ('700.0', '580.0'), '35': ('685.0', '595.0'), '36': ('685.0', '610.0'), 

In [8]:
def computeWeight(First, Second, Type):
    x1 = float(First[0])
    x2 = float(Second[0])
    y1 = float(First[1])
    y2 = float(Second[1])
    if Type == "EUC_2D":
        return sqrt((x1-x2)**2 + (y1-y2)**2)
    if Type == "GEO":
        PI = 3.141592
        R = 6378.388 
        deg = int(x1)
        minn = x1 - deg
        lat1 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        deg = int(x2)
        minn = x2 - deg
        lat2 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        deg = int(y1)
        minn = y1 - deg
        lon1 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        deg = int(y2)
        minn = y2 - deg
        lon2 = PI * (deg + 5.0 * minn/ 3.0) / 180.0
        dlon = cos(lon2 - lon1)
        dlat = cos(lat2 - lat1)
        slat = cos(lat2 + lat1)
        di = R * acos( 0.5*((1.0+dlon)*dlat - (1.0-dlon)*slat) ) + 1.0
        return di

In [9]:
print(Coordinates['1'],Coordinates['2'])
print(computeWeight(Coordinates['1'],Coordinates['2'],EDGE_WEIGHT_TYPE))

('565.0', '575.0') ('25.0', '185.0')
666.1080993352356


In [10]:
def getKey(sdict):
    key_list = list(sdict.keys())
    return key_list

def makeGraph(Coordinates):
    keys = getKey(Coordinates)
    matrix = []
    for i in range(len(keys)):
        u = keys[i]
        for j in range(i,len(keys)):
            v = keys[j]
            if v == u:
                continue
            w = computeWeight(Coordinates[u],Coordinates[v],EDGE_WEIGHT_TYPE) # weight computed for edge(u,v)
            matrix.append((u,v,w))
    return keys, matrix

def makeDict(Coordinates):
    keys = getKey(Coordinates)
    dictionary = {}
    for i in range(len(keys)):
        u = keys[i]
        for j in range(i,len(keys)):
            v = keys[j]
            if v == u:
                continue
            w = computeWeight(Coordinates[u],Coordinates[v],EDGE_WEIGHT_TYPE) # weight computed for edge(u,v)
            dictionary[(u,v)] = (u,v,w)
    return dictionary

In [11]:
"""
Nearest Neighbor Algorithm
sort the edges
pich the first node
"""
#  {"1": [(2,w), (3,w), ...]...}
# creating adjacency matrix with sorted weights
def make_set_nn(all_edges):
    nn_weights = {}
    for e in all_edges:
        if e[0] not in nn_weights:
            nn_weights[e[0]] = []
        # add edges to the adjacency list    
        if e[1] not in nn_weights:
            nn_weights[e[1]] = []
        nn_weights[e[0]].append((e[1], e[2]))
        nn_weights[e[1]].append((e[0], e[2]))
    # sort weights
    for vertice in nn_weights:
        nn_weights[vertice] = sorted(nn_weights[vertice], key=lambda el: el[1])
        
    return nn_weights



def nearest_neighbour(vertice, matrix_nn, visited, path):
    # if we saw the vertex before, return
    if vertice not in matrix_nn or len(matrix_nn[vertice]) == 0:
        return 
    
    # get the lightest edge
    lightest = matrix_nn[vertice].pop(0)    
    # if we saw the neighbour before, go forward
    while lightest[0] in visited:
        if len(matrix_nn[vertice]) == 0:
            return
        lightest = matrix_nn[vertice].pop(0)
    
    # found the unseen one, remove fron adj_list
    matrix_nn.pop(vertice)
    # append the edge to a path
    path.append((vertice , lightest[0], lightest[1]))
    # populate the visited set
    visited.add(lightest[0])
    # call recursively for another vertice
    nearest_neighbour(lightest[0], nn_mtrx, visited, path)

def nnWeight(solution):
    return sum([edge[2] for edge in solution])

In [12]:
nnSol = []
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates  = parse(file)
    vertices, edges = makeGraph(Coordinates)
    # define source
    source = vertices[0]
    # create adjacency matrix
    nn_mtrx = make_set_nn(edges)
    gc.disable()
    start_time = perf_counter_ns()
    path = []
    # save the adjacency for the source vertex
    source_list = nn_mtrx[source]
    visited = set([source])
    nearest_neighbour(source, nn_mtrx, visited, path)
    # create a cycle
    last_edge = [s for s in source_list if s[0] == path[-1][0]][0]
    cycle = path + [(last_edge[0], source, last_edge[1])]
    end_time = perf_counter_ns()
    time = end_time - start_time
    gc.enable()
    weight = nnWeight(cycle)
    print("did",NAME,"in",time,"ns with a weight of:",weight)
    nnSol.append((NAME,weight,time))

did berlin52 in 1010100 ns with a weight of: 8955.61046089277
did burma14 in 157300 ns with a weight of: 3431.5835605588695
did ch150 in 3241700 ns with a weight of: 8231.728753874797
did d493 in 34709100 ns with a weight of: 43718.5982437295
did dsj1000 in 236845700 ns with a weight of: 24491641.416672677
did eil51 in 12090700 ns with a weight of: 486.50505971531834
did gr202 in 9034700 ns with a weight of: 51233.510002105475
did gr229 in 5558700 ns with a weight of: 163017.6141288092
did kroA100 in 1492600 ns with a weight of: 26449.359516957113
did kroD100 in 1732900 ns with a weight of: 27075.76912745193
did pcb442 in 21130000 ns with a weight of: 65502.30967319354
did ulysses16.tsp in 525700 ns with a weight of: 8183.059053043473
did ulysses22.tsp in 106800 ns with a weight of: 8792.026234973438


### Efficient Kruskal 

In [13]:
# Graph object
class Graph:
    def __init__(self, V, E, num_V, num_E):
        self.V = V
        self.E = E
        self.num_V = num_V
        self.num_E = num_E

In [14]:
"""
Kruskal Efficient Agorithm
sort the edges
make a list of vertices
get the first edge
find the parent of the vertices in the edge(u,v)
if there is no loop union
"""
class Kruskal_Efficient:
    
    def __init__(self, graph):
        self.graph = graph
        self.sets = {} # set of vertices
        self.MST = [] # Minimum Spanning Tree
    
    
    # make a set of vertices
    def make_sets(self):
        for v in self.graph.V:
            self.sets[v] = [v]
        
    
    
    # union the subsets which the vertices are not in the same sets
    def union(self, u_prnt, v_prnt):
        # get the size of two elements and append the vertices to the bigger one
        if (len(self.sets.get(u_prnt)) >= len(self.sets.get(v_prnt))):
            self.sets[u_prnt].extend(self.sets[v_prnt])
            self.sets.pop(v_prnt)
        
        # append the list of vertices of parent u to v
        else:
            self.sets[v_prnt].extend(self.sets[u_prnt])
            self.sets.pop(u_prnt)
    
    
    
    # find the parent of u and v vertices and return the parents
    def find_parent(self, u, v, items):
        u_key = v_key = 0
        for item in list(items):
            # item[0] is the key in dictionary
            # item[1] is the values in the dictionary
            key, value = item[0], item[1] 
            #if_true = all(x in value for x in[u, v]) # check if both u and v are in the same set
            # check the vertices in the value list and return the key as the parent of the vertex
            if u in value:
                u_key = item[0]
            if v in value:
                v_key = item[0]
            if u_key and v_key:
                break
        return (u_key, v_key)
    
    
    def connecting(self, mst, all_edges):
        leng = len(mst)
        leng -= 1
        first_node = mst[0][0]
        last_node = mst[leng][1]
        for e in all_edges:
            u,v,w = e
            if (first_node == u and last_node == v):
                mst.append(e)
            if (first_node == v and last_node == u):
                mst.append(e) 
        return mst
    
    
    
    # make the MST tree
    def execute(self):
#         mst_weight = 0
#         lenv = len(self.graph.V) # number of vertices
        
        # sorting the edges based on the wight of the edges    
        E = sorted(self.graph.E, key = lambda m: m[2])
#         print("this: ", E)
        
        self.make_sets() # make a set of vertices
        # make a list of sets of key and value pairs to iterate through them
        items = self.sets.items()
        for e in E:
            # check if number of edges in MST are less than  nodes are 
            if((len(self.MST)+1) <  int(self.graph.num_E)):
                u, v, w = e
                u_parent, v_parent = self.find_parent(u, v, items)
                # if the vertices(u,v) are not in the same sets
                if (u_parent != v_parent):
                    self.union(u_parent, v_parent)
                    # add the edge to the MST[]
                    self.MST.append(e)
            # if the MST is completed, stop looping through the edges
            else:
                break
                
        self.MST = self.connecting(self.MST, E)
        
        return self.MST

    
    # calculate the final weight of the MST
    def MSTweight_EK(self):
        sum = 0
        for (u ,v, w) in self.MST:
            sum = sum + w
        return sum

In [15]:
twoAppSol = []
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates = parse(file)
    vertices, edges = makeGraph(Coordinates)
    num_V_E = [len(vertices), len(edges)]
    gc.disable()
    start_time = perf_counter_ns()
    graph = Graph(vertices, edges, num_V_E[0], num_V_E[1])
    algo = Kruskal_Efficient(graph)
    result = algo.execute()
    weight = algo.MSTweight_EK()
    end_time = perf_counter_ns()
    time = end_time - start_time
    gc.enable()
    print("did",NAME,"in",time,"ns with a weight of:",weight)
    twoAppSol.append((NAME,weight,time))

did berlin52 in 51103100 ns with a weight of: 6458.094351322505
did burma14 in 356200 ns with a weight of: 2354.182307431727
did ch150 in 87635300 ns with a weight of: 6152.31044207293
did d493 in 2138747900 ns with a weight of: 31549.748274636288
did dsj1000 in 20121555200 ns with a weight of: 16581592.843637954
did eil51 in 40908800 ns with a weight of: 413.2872980055961
did gr202 in 216743300 ns with a weight of: 36869.14775735362
did gr229 in 226096300 ns with a weight of: 123897.01378037801
did kroA100 in 25255500 ns with a weight of: 21725.272085037737
did kroD100 in 24503700 ns with a weight of: 21430.38905104946
did pcb442 in 1423865200 ns with a weight of: 49868.10016024597
did ulysses16.tsp in 3939600 ns with a weight of: 6415.9884957518125
did ulysses22.tsp in 751600 ns with a weight of: 6447.313247767479


# Random Insertion

In [16]:
def findClosest(seen, alledges):
    sWeight = 999999
    for (u,v,w) in alledges:
        for place in seen:
            if u == place or v == place:
                if sWeight > w:
                    sWeight = w
                    smallest = (u,v,w)
    if smallest[0] == place:
        return smallest[1], smallest
    return smallest[0], smallest


def chooseRandom(unseen):
    randPlace = choice(unseen)
    return randPlace

def weightCheck(rand, seen, edges, unseen):
    bestWeight = float("inf")
    ijk = []
    ik = None
    kj = None
    ij = None
    for old1 in seen:
        cpseen = seen.copy()
        cpseen.remove(old1)
        for old2 in cpseen:
                if (old1,rand) in edges:
                    ik = edges[(old1,rand)]
                elif (rand,old1) in edges:
                    ik = edges[(rand,old1)]
                    
                if (old2,rand) in edges:
                    kj = edges[(old2,rand)]
                elif (rand,old2) in edges:
                    kj = edges[(rand,old2)]
                    
                    
                if (old1,old2) in edges:
                    ij = edges[(old1,old2)]
                elif (old2,old1) in edges:
                    ij = edges[(old2,old1)]
                    
                weight = ik[2] + kj[2] - ij[2]
                if bestWeight > weight:
                    currentBest = [ik,kj,ij]
                ik = None
                kj = None
                ij = None
    return currentBest[2],currentBest[0],currentBest[1]

def totalWeight(solution):
    return sum([edge[2] for edge in solution])

def addTheLastOne(solu, rand):
    if ('1',rand) in edgeDict:
        toadd = edgeDict[('1',rand)]
    elif (rand,'1') in edgeDict:
        toadd = edgeDict[(rand,'1')]
    return toadd

In [17]:
RanInsSol = []
for file in os.listdir():
    NAME , EDGE_WEIGHT_TYPE , DIMENSION , Coordinates = parse(file)
    edgeDict = makeDict(Coordinates)
    vertices, edges = makeGraph(Coordinates)
    num_V_E = [len(vertices), len(edges)]
    gc.disable()
    start_time = perf_counter_ns()
    solution = []
    seenPlaces = ['1'] #our starting node
    closestPlace, newEdge = findClosest(seenPlaces,edges)
    seenPlaces.append(closestPlace)
    solution.append(newEdge)
    x = seenPlaces
    y = vertices
    unseen = list(set(y) - set(x))
    while unseen:    
        randomPlace = chooseRandom(unseen)
        toRemove, toAdd1, toAdd2 = weightCheck(randomPlace, seenPlaces, edgeDict, unseen)
        solution.remove(toRemove)
        solution.append(toAdd1)
        solution.append(toAdd2)
        seenPlaces.append(randomPlace)
        x = seenPlaces
        y = vertices
        unseen = list(set(y) - set(x))
    toAdd = addTheLastOne(solution,randomPlace)
    solution.append(toAdd)
    end_time = perf_counter_ns()
    gc.enable()
    time = end_time - start_time
    weight = totalWeight(solution)
    RanInsSol.append((NAME,weight,time))
    print("did",NAME,"in",time,"ns with a weight of:",weight)

did berlin52 in 167569400 ns with a weight of: 28549.824140941604
did burma14 in 3048400 ns with a weight of: 7366.537177800381
did ch150 in 2195855000 ns with a weight of: 54711.58209593793
did d493 in 105771093300 ns with a weight of: 461458.43034749635
did dsj1000 in 752737846900 ns with a weight of: 563256544.0348397
did eil51 in 75167000 ns with a weight of: 1590.1436118913316
did gr202 in 4487955500 ns with a weight of: 272495.46319980174
did gr229 in 6759444100 ns with a weight of: 1391955.5133572305
did kroA100 in 499140100 ns with a weight of: 183392.71981091244
did kroD100 in 481650200 ns with a weight of: 165862.69504292685
did pcb442 in 67060898100 ns with a weight of: 781741.3803558962
did ulysses16.tsp in 2660700 ns with a weight of: 11168.879641739319
did ulysses22.tsp in 11020300 ns with a weight of: 18052.56942392819
